# Traitement des données météo de 2005 à 2009

In [14]:
import pandas as pd
import numpy as np

## Import du dataset

In [15]:
data_meteo = pd.read_csv('../data/decad_2010_2019.csv')
data_meteo.head()

C:\Users\mathi\AppData\Local\Temp\ipykernel_6996\2115385618.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data_meteo = pd.read_csv('../data/decad_2010_2019.csv')


,__id,LAMBX,LAMBY,DATE,DECADE,PRENEI_DECAD,PRELIQ_DECAD,PRETOTD_DECAD,T_DECAD,EVAP_DECAD,ETP_DECAD,PE_DECAD,SWI_DECAD,SSWI_DECAD,ECOULEMENT_DECAD,DRAINC_DECAD,RUNC_DECAD
0,1,600,24010,201001,1,1.2,23.4,24.6,2.5,15.2,7.4,9.4,0.951,0.52,1.2,21.4,2.1
1,4,600,24010,201002,1,0.0,27.0,27.0,6.5,14.5,10.5,12.5,0.918,0.27,0.0,15.3,2.4
2,7,600,24010,201003,1,0.0,0.5,0.5,5.0,16.4,19.9,-15.9,0.979,1.54,0.0,30.4,0.0
3,10,600,24010,201004,1,0.0,28.4,28.4,9.0,23.8,27.7,4.6,0.855,0.49,0.0,9.5,2.5
4,13,600,24010,201005,1,0.0,16.2,16.2,9.7,17.1,27.3,-0.9,0.669,-0.76,0.0,3.1,0.9


In [16]:
# Affichage des infos
data_meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187063 entries, 0 to 1187062
Data columns (total 17 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   __id              1187063 non-null  object
 1   LAMBX             1187063 non-null  object
 2   LAMBY             1187063 non-null  object
 3   DATE              1187063 non-null  object
 4   DECADE            1187063 non-null  object
 5   PRENEI_DECAD      1187063 non-null  object
 6   PRELIQ_DECAD      1187063 non-null  object
 7   PRETOTD_DECAD     1187063 non-null  object
 8   T_DECAD           1187063 non-null  object
 9   EVAP_DECAD        1187063 non-null  object
 10  ETP_DECAD         1187063 non-null  object
 11  PE_DECAD          1187063 non-null  object
 12  SWI_DECAD         1187063 non-null  object
 13  SSWI_DECAD        1077740 non-null  object
 14  ECOULEMENT_DECAD  1187063 non-null  object
 15  DRAINC_DECAD      1187063 non-null  object
 16  RUNC_DECAD        

In [17]:
# 1) Forcer T_DECAD en type numérique (float), en remplaçant les valeurs non-convertibles par NaN
data_meteo['T_DECAD'] = pd.to_numeric(
    data_meteo['T_DECAD'],
    errors='coerce'
)

# 2) Grouper par DATE et calculer la moyenne
df_meteo = (
    data_meteo
      .groupby('DATE', as_index=False)['T_DECAD']
      .mean()
      .rename(columns={'T_DECAD':'temp_AVG', 'DATE': 'Date'})
)

df_meteo.head()


,Date,temp_AVG
0,201001,-2.016865
1,201002,2.039407
2,201003,1.422099
3,201004,7.812351
4,201005,8.784769


In [18]:
# Calcul du nombre de valeurs manquantes
df_meteo.isna().sum()

Date        0
temp_AVG    1
dtype: int64

In [19]:
# Calcul du nombre de NaN par colonne
df_meteo.isna().sum(axis=0)

Date        0
temp_AVG    1
dtype: int64

In [20]:
# Création d'une colonne Année à partir des 4 premiers caractères de la colonne Date
col_date = 'Date' if 'Date' in df_meteo.columns else 'DATE'

# Conversion en string, en remplaçant les NaN par une chaîne vide
df_meteo[col_date] = df_meteo[col_date].fillna('').astype(str)

# Extraction des 4 premiers caractères et forcer en numérique
# Les non-numériques deviendront NaN
df_meteo['Année'] = pd.to_numeric(
    df_meteo[col_date].str[:4],
    errors='coerce'
)
# 
df_meteo = df_meteo.dropna(subset=['Année'])
df_meteo['Année'] = df_meteo['Année'].astype(int)

df_meteo.head()

,Date,temp_AVG,Année
0,201001,-2.016865,2010
1,201002,2.039407,2010
2,201003,1.422099,2010
3,201004,7.812351,2010
4,201005,8.784769,2010


In [21]:
col_date = 'Date' if 'Date' in df_meteo.columns else 'DATE'

# 2) S’assurer que c’est bien du string (et pas de NaN)
df_meteo[col_date] = df_meteo[col_date].fillna('').astype(str)

# 3) Extraire les deux derniers caractères pour le mois, convertir en numérique
df_meteo['Mois'] = pd.to_numeric(
    df_meteo[col_date].str[-2:],   # ex. "200501" → "01"
    errors='coerce'                # invalides → NaN
)

# 4) (Optionnel) filtrer/vider les lignes où l’extraction a échoué
df_meteo = df_meteo.dropna(subset=['Mois'])
df_meteo['Mois'] = df_meteo['Mois'].astype(int)

df_meteo.head()

,Date,temp_AVG,Année,Mois
0,201001,-2.016865,2010,1
1,201002,2.039407,2010,2
2,201003,1.422099,2010,3
3,201004,7.812351,2010,4
4,201005,8.784769,2010,5


In [22]:
# Création d'une colonne Nom du Mois
mois_fr = {
    1:  'Janvier',
    2:  'Février',
    3:  'Mars',
    4:  'Avril',
    5:  'Mai',
    6:  'Juin',
    7:  'Juillet',
    8:  'Août',
    9:  'Septembre',
    10: 'Octobre',
    11: 'Novembre',
    12: 'Décembre'
}

# 2) Création de la colonne Nom_Mois
df_meteo['Nom_Mois'] = df_meteo['Mois'].map(mois_fr)

df_meteo.head()

,Date,temp_AVG,Année,Mois,Nom_Mois
0,201001,-2.016865,2010,1,Janvier
1,201002,2.039407,2010,2,Février
2,201003,1.422099,2010,3,Mars
3,201004,7.812351,2010,4,Avril
4,201005,8.784769,2010,5,Mai


In [23]:
# Affichage des types
df_meteo.dtypes

Date         object
temp_AVG    float64
Année         int64
Mois          int64
Nom_Mois     object
dtype: object

In [24]:
# Export en CSV
df_meteo.to_csv('../staged_data/decad_2010_2019_meteo.csv', index=False)